In [6]:
#make imports
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [7]:
#connect to the netflix database
#attempt connection to a database on the database server
def create_db_connection():
    connection = None
    try:
        connection = mysql.connector.connect(
            host="localhost",
            user="root",
            passwd="*****",
            database="netflix"
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

#create connection variable
connection = create_db_connection()

MySQL Database connection successful


In [8]:
#create functions to interact with database
#function to execute a query on the database connection
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

#function to execute a query multiple times with lists of values where 'sql' is the sql command and val is the lists of values to be entered with the query
#ie
    #sql = '''
    #    INSERT INTO teacher (teacher_id, first_name, last_name, language_1, language_2, dob, tax_id, phone_no)
    #    VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
    #    '''
    #
    #val = [
    #    (7, 'Hank', 'Dodson', 'ENG', None, '1991-12-23', 11111, '+491772345678'), 
    #    (8, 'Sue', 'Perkins', 'MAN', 'ENG', '1976-02-02', 22222, '+491443456432')
    #]
def execute_list_query(connection, sql, val):
    cursor = connection.cursor()
    try:
        cursor.executemany(sql, val)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

#function to execute a query on the database that will return information as a list of lists
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        #turn the result into a list of lists
        ret_list = []
        for element in result:
            element = list(element)
            ret_list.append(element)
        return ret_list
    except Error as err:
        print(f"Error: '{err}'")



In [9]:
#turn the tables into readable dataframes
def df_series():
    #retrieve the table from the database
    sql = """
    SELECT * FROM Series;
    """
    results = read_query(connection, sql)

    #turn the results into a dataframe
    columns = ['id', 'Title', 'Movie']
    df = pd.DataFrame(results, columns=columns)
    df.set_index('id')

    return df

def df_users():
    #retrieve the table from the database
    sql = """
    SELECT * FROM Users;
    """
    results = read_query(connection, sql)

    #turn the results into a dataframe
    columns = ['id', 'Name']
    df = pd.DataFrame(results, columns=columns)
    df.set_index('id')

    return df

def df_devices():
    #retrieve the table from the database
    sql = """
    SELECT * FROM Devices;
    """
    results = read_query(connection, sql)

    #turn the results into a dataframe
    columns = ['id', 'Name']
    df = pd.DataFrame(results, columns=columns)
    df.set_index('id')

    return df

def df_sessions():
    #retrieve the table from the database
    sql = """
    SELECT * FROM Sessions;
    """
    results = read_query(connection, sql)

    #turn the results into a dataframe
    columns = ['id', 'User_id', 'Series_id', 'Title', 'Start_Time', 'Duration', 'Device_id']
    df = pd.DataFrame(results, columns=columns)
    df.set_index('id')

    return df

In [10]:
print(df_series().to_string())

        id                                                     Title  Movie
0     1000                                                    Avatar      0
1     1003                                 Avatar The Last Airbender      0
2     1029                         Glass Onion: A Knives Out Mystery      1
3     1030                                                 ONE PIECE      0
4     1031                                    Leave the World Behind      1
5     1032                               The Super Mario Bros. Movie      1
6     1033                                                The Batman      1
7     1034                                   The Cloverfield Paradox      1
8     1035                                   Scott Pilgrim Takes Off      0
9     1043                                      Arrested Development      0
10    1075                                             LEGO Bionicle      0
11    1088                                           Stranger Things      0
12    1137  

In [13]:
#find all movies
sql = """
SELECT Title
FROM Series
WHERE Movie = 1
ORDER BY Title;
"""
print(pd.DataFrame(read_query(connection,sql)).to_string())

                                                     0
0                                             365 Days
1                                   365 Days: This Day
2                                        6 Underground
3                                 A Bad Moms Christmas
4                               A California Christmas
5                               A Castle For Christmas
6                                   A Christmas Prince
7                   A Christmas Prince: The Royal Baby
8                A Christmas Prince: The Royal Wedding
9                   A Cinderella Story: Christmas Wish
10                                    A Loser Presents
11                                   A Man Called Otto
12                                      A Silent Voice
13                            A Very Country Christmas
14                             A Very Murray Christmas
15                             Abducted in Plain Sight
16                                          About Time
17        

In [18]:
#find all sessions for user Alec
sql = """
SELECT Sessions.id, Sessions.Device_id, Sessions.Title, Sessions.Series_id, Sessions.Start_Time, Sessions.Duration 
FROM Users
LEFT JOIN Sessions
ON Users.id = Sessions.User_id
WHERE Users.Name = 'Alec';
"""
print(pd.DataFrame(read_query(connection,sql)).to_string())

          0     1                                                                                                                              2     3                   4               5
0     10000   100                                                            Avatar: The Last Airbender: Book 1: The Fortune-Teller (Episode 14)  1000 2024-03-11 05:30:06 0 days 00:23:43
1     10001   100                                                               Avatar: The Last Airbender: Book 1: The Blue Spirit (Episode 13)  1000 2024-03-11 05:19:23 0 days 00:10:39
2     10002   102                                                               Avatar: The Last Airbender: Book 1: The Blue Spirit (Episode 13)  1000 2024-03-07 22:23:35 0 days 00:10:46
3     10003   103                                                                      Avatar The Last Airbender: Season 1: Warriors (Episode 2)  1003 2024-03-07 20:49:49 0 days 00:21:10
4     10004   103                                                

In [20]:
#find all series watched by user Alec
sql = """
SELECT DISTINCT Series.Title 
FROM Series
INNER JOIN Sessions
ON Series.id = Sessions.Series_id
INNER JOIN Users
ON User_id = Users.id
WHERE Users.Name = 'Alec'
ORDER BY Series.Title;
"""
print(pd.DataFrame(read_query(connection, sql)).to_string())

                                                    0
0                                      13 Reasons Why
1                                    A Loser Presents
2                                      A Silent Voice
3                             A Very Murray Christmas
4                     Alice Through the Looking Glass
5                                Ant-Man and the Wasp
6                                Arrested Development
7                                               Arrow
8                           Atlantis: The Lost Empire
9                                     Attack on Titan
10                        Austin Powers in Goldmember
11        Austin Powers: International Man of Mystery
12              Austin Powers: The Spy Who Shagged Me
13                                             Avatar
14                          Avatar The Last Airbender
15                                        Bellydancer
16                             Beyond Stranger Things
17                          

In [22]:
#find all device names for user Alec
sql = """
SELECT DISTINCT Devices.Name
FROM Devices 
LEFT JOIN Sessions
ON Devices.id = Sessions.Device_id
LEFT JOIN Users
ON Sessions.User_id = Users.id
WHERE Users.Name = 'Alec'
ORDER BY Devices.Name;
"""
print(pd.DataFrame(read_query(connection, sql)).to_string())

                                               0
0    Amazon Fire TV Stick 2020 + Streaming Stick
1                        Amazon TVAFTBL Smart TV
2                            Apple iPhone 11 Pro
3                                Apple iPhone 6s
4                          Apple iPhone 7 (CDMA)
5                            Chrome PC (Cadmium)
6              Comcast X1 ARM WITH SAGE MVPD STB
7             Comcast X1 MIPS WITH SAGE MVPD STB
8                   Edge OSS - Windows (Cadmium)
9                           Firefox PC (Cadmium)
10                             FireTV Stick 2014
11                             FireTV Stick 2016
12             Google Chromecast streaming stick
13                            Microsoft Xbox 360
14                            Microsoft Xbox One
15                          Microsoft Xbox One X
16  Netflix Windows App - Cadmium Windows Mobile
17                      Roku Fruitland Sound Bar
18     Sony Sony SONY BD 2017 CXE Blu-ray Player
19           Vizio D